In [60]:
import numpy as np
import random

class QuantumParticle:
    def __init__(self, position, velocity):
        self.position = position
        self.velocity = velocity
        self.pbest_position = position
        self.pbest_fitness = -float("inf")
        self.fitness = -float("inf")

    def update_position(self, c1, c2, w, number_of_mecs, gbest_position):
        r1 = random.random()
        r2 = random.random()

        phi1 = (np.pi * r1) - (np.pi / 2)
        phi2 = (np.pi * r2) - (np.pi / 2)

        chi1 = np.cos(phi1)
        chi2 = np.cos(phi2)

        self.velocity = w * self.velocity + c1 * chi1 * (self.pbest_position - self.position) + c2 * chi2 * (gbest_position - self.position)
        self.position = int(np.clip(self.position + self.velocity, 0, number_of_mecs - 1))

    def calculate_fitness(self, transmission_rates, task_data, mec_computing_capacity):
        offloading_time = np.array(task_data) / transmission_rates[self.position]
        execution_time = np.array(task_data) / mec_computing_capacity
        total_delay = offloading_time + execution_time
        self.fitness = np.mean(1 / total_delay)

class QuantumParticleSwarmOptimization:
    def __init__(self, transmission_rates, number_of_devices, task_data, maximum_tolerable_delay, device_computing_capacity, number_of_mecs, mec_computing_capacity, transmission_power):
        self.transmission_rates = transmission_rates
        self.task_data = task_data
        self.number_of_mecs = number_of_mecs
        self.mec_computing_capacity = mec_computing_capacity

        self.particles = []
        for _ in range(len(number_of_devices)):
            position = np.random.randint(0, self.number_of_mecs)
            velocity = np.random.uniform(-1, 1)
            self.particles.append(QuantumParticle(position, velocity))

        self.gbest_position = self.particles[0].position
        self.gbest_fitness = -float("inf")

    def optimize(self, c1, c2, w, max_iterations):
        for iteration in range(max_iterations):
            for particle in self.particles:
                particle.update_position(c1, c2, w, self.number_of_mecs, self.gbest_position)
                particle.calculate_fitness(self.transmission_rates, self.task_data, self.mec_computing_capacity)

                if particle.fitness > particle.pbest_fitness:
                    particle.pbest_position = particle.position
                    particle.pbest_fitness = particle.fitness

                if particle.fitness > self.gbest_fitness:
                    self.gbest_position = particle.position
                    self.gbest_fitness = particle.fitness

        return self.gbest_position

# Set the parameters
transmission_rates = [500, 1000, 1500, 2000, 2500, 3000]
number_of_devices = [50, 100, 150, 200, 250, 300]
task_data = [5, 10, 15, 20, 25, 30, 35, 40, 45]
maximum_tolerable_delay = 0.5
device_computing_capacity = 0.5
number_of_mecs = 10
mec_computing_capacity = 4
transmission_power = 0.5

# Initialize the QPSO algorithm
qpso = QuantumParticleSwarmOptimization(transmission_rates, number_of_devices, task_data, maximum_tolerable_delay, device_computing_capacity, number_of_mecs, mec_computing_capacity, transmission_power)

# Optimize the task offloading strategy
gbest = qpso.optimize(c1=2, c2=2, w=0.7, max_iterations=100)

# Print the optimal task offloading strategy
print(gbest)



IndexError: list index out of range

In [59]:
import numpy as np

# Parameters
n_iterations = 100
n_device_options = [50, 100, 150, 200, 250, 300]
n_mecs = 10
mec_capacity = 4
transmission_power = 0.5
mec_voltage = 1.0
device_voltage = 1.0
task_data = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45])
transmission_rate = np.array([500, 1000, 1500, 2000, 2500, 3000]) * 1e-3

# Break down objective calculations
def energy_consumption(position):
    # energy consumed per device to transmit its task
    per_device_energy = transmission_power * device_voltage * np.sum(task_data * position)
    return per_device_energy


def completion_time(position):
    # Average transmission rate for simplicity
    avg_transmission_rate = np.mean(transmission_rate)
    
    return np.sum(task_data / (avg_transmission_rate * position))


# Objective function combining both
def objective_function(position):
    # Combine using some weight (you can adjust this)
    weight_energy = 0.5
    weight_time = 0.5
    
    return weight_energy * energy_consumption(position) + weight_time * completion_time(position)

class QPSO:
    def __init__(self, dim, n_particles, iterations):
        self.dim = dim
        self.n_particles = n_particles
        self.iterations = iterations
        
        # Initialize positions and velocities
        self.positions = np.random.uniform(0, 1, (n_particles, dim))
        self.velocities = np.random.uniform(-0.1, 0.1, (n_particles, dim))

        # Initialize pbest and gbest
        self.pbest_positions = self.positions.copy()
        self.pbest_values = np.array([float('inf')] * n_particles)
        self.gbest_position = np.random.uniform(0, 1, dim)
        self.gbest_value = float('inf')
        
        # Constants for QPSO, you can adjust these or initialize them externally
        self.alpha = 0.5  # inertia
        self.beta = 1.5  # personal attraction
        self.gamma = 1.5  # global attraction


        
    def update(self):
        for i in range(self.n_particles):
            # Update velocity
            self.velocities[i] = (self.alpha * self.velocities[i] + 
                                self.beta * np.random.uniform(0, 1, self.dim) * (self.pbest_positions[i] - self.positions[i]) +
                                self.gamma * np.random.uniform(0, 1, self.dim) * (self.gbest_position - self.positions[i]))

            # Update position
            self.positions[i] += self.velocities[i]

            # Make sure position values are non-negative
            self.positions[i] = np.maximum(self.positions[i], 0)
           
            
            # Update pbest
            current_value = objective_function(self.positions[i])
            if current_value < objective_function(self.pbest_positions[i]):
                self.pbest_positions[i] = self.positions[i].copy()

            
    def optimize(self):
        for _ in range(self.iterations):
            self.update()
            
        return self.gbest_value, self.gbest_position


# Instantiate and run QPSO for each device setting
results = {}

for n_device in n_device_options:
    qpso = QPSO(dim=len(task_data), n_particles=n_device, iterations=n_iterations)
    best_value, best_position = qpso.optimize()
    mean_energy = energy_consumption(best_position)
    mean_time = completion_time(best_position) / n_device
    results[n_device] = (best_value, best_position, mean_energy, mean_time)

for n_device, (value, position, energy, time) in results.items():
    print(f"For {n_device} devices -> Best value: {value}")
    print(f"Mean Energy Consumption: {energy}")
    print(f"Mean Task Completion Time: {time}\n")


C:\Users\YANSILIYU\AppData\Local\Temp\ipykernel_13704\3084376228.py:25: RuntimeWarning: divide by zero encountered in divide
  return np.sum(task_data / (avg_transmission_rate * position))


For 50 devices -> Best value: inf
Mean Energy Consumption: 53.561535311709505
Mean Task Completion Time: 7.20873049096158

For 100 devices -> Best value: inf
Mean Energy Consumption: 47.25046826148599
Mean Task Completion Time: 19.882162268761405

For 150 devices -> Best value: inf
Mean Energy Consumption: 60.652744693254185
Mean Task Completion Time: 97.13831437298099

For 200 devices -> Best value: inf
Mean Energy Consumption: 47.92483169636248
Mean Task Completion Time: 9.355334908917621

For 250 devices -> Best value: inf
Mean Energy Consumption: 66.28648411048836
Mean Task Completion Time: 1.5842902141059239

For 300 devices -> Best value: inf
Mean Energy Consumption: 50.411567216917454
Mean Task Completion Time: 1.353563309896352

